<a href="https://colab.research.google.com/github/Aksinhaa/ColabFold/blob/main/Genomics_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<video src="https://raw.githubusercontent.com/PoODL-CES/PoODL_NGS_workshop.io/main/Nature%20of%20Science.mp4" controls autoplay loop width="400" align="right"></video>

## Genomics Learning Workshop: NGS Analysis Pipeline

Easy-to-follow workflow for processing Illumina whole-genome resequencing reads for population genomics. Steps include trimming, mapping, sorting, variant calling, variant filtering, PCA, and ADMIXTURE analysis. For more details, see the bottom of this notebook and visit the GitHub repository.
Repository link: [Genomics Learning Workshop](https://github.com/PoODL-CES/Genomics_learning_workshop)

This workshop introduces:
- Handling raw FASTQ files
- Performing quality control and trimming
- Mapping reads to a reference genome
- Calling and filtering variants
- Running PCA and ADMIXTURE for population analysis


In [5]:
#@title Install system-level NGS bioinformatics tools
%%time

# Install samtools, fastqc, vcftools using apt-get (stable)
!apt-get update -qq
!apt-get install -y samtools fastqc vcftools

# Install cutadapt (requirement for Trim Galore, also installed via conda later)
!pip install --quiet cutadapt

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  at-spi2-core default-jre default-jre-headless fonts-dejavu-core
  fonts-dejavu-extra gsettings-desktop-schemas libapache-pom-java
  libargs4j-java libatk-bridge2.0-0 libatk-wrapper-java
  libatk-wrapper-java-jni libatk1.0-0 libatk1.0-data libatspi2.0-0
  libcommons-compress-java libcommons-io-java libcommons-jexl2-java
  libcommons-lang3-java libcommons-logging-java libcommons-math3-java
  libcommons-parent-java libfindbin-libs-perl libhts3 libhtscodecs2
  libhtsjdk-java libjbzip2-java libjson-simple-java libngs-java libngs-sdk-dev
  libngs-sdk2 libsis-base-java libsis-base-jni libsis-jhdf5-java
  libsis-jhdf5-jni libsnappy-java libsnappy-jni libxcomp

In [6]:
# Miniconda installation and environment setup for Colab NGS Workshop

# Download and install Miniconda (skip if already installed)
!wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
!bash miniconda.sh -b -p /usr/local/miniconda

import sys, os
sys.path.append('/usr/local/miniconda/lib/python3.8/site-packages')
os.environ['PATH'] = "/usr/local/miniconda/bin:" + os.environ['PATH']

# Explicitly clear potentially problematic environment variables from Python's os.environ
if 'CONDA_PREFIX' in os.environ:
    del os.environ['CONDA_PREFIX']
if 'CONDA_ENVS_PATH' in os.environ:
    del os.environ['CONDA_ENVS_PATH']

# Accept ToS for main and R conda channels
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

# Create new conda environment for your workshop if it doesn't exist
!if ! conda info --envs | grep -w 'workshop_ngs'; then \
    conda create -y -n workshop_ngs python=3.7; \
else \
    echo "Environment 'workshop_ngs' already exists, skipping creation."; \
fi

# Install necessary bioinformatics tools into the environment
!conda install -y -n workshop_ngs -c bioconda -c conda-forge trim-galore samtools fastqc vcftools gatk4

# Verify tool versions inside activated environment in one shell session
!bash -c "source /usr/local/miniconda/bin/activate workshop_ngs && \
          trim_galore --version && samtools --version && fastqc --version && \
          vcftools --version && gatk --version"

PREFIX=/usr/local/miniconda
Unpacking bootstrapper...
Unpacking payload...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Miniconda3: /usr/local/miniconda
accepted Terms of Service for https://repo.anaconda.com/pkgs/main
accepted Terms of Service for https://repo.anaconda.com/pkgs/r
Jupyter detected...
2 channel Terms of Service accepted
Retrieving notices: - \ | / - \ | / - done
Channels:
 - defaults
Platform: linux-64
Solving environment: | done

## Package Plan ##

  environment location: /usr/local/miniconda/envs/workshop_ngs

  added / updated specs:
    - python=3.7



To activate the `workshop_ngs` environment for specific commands and ensure its tools are used, you can wrap your commands within a `bash -c "source /usr/local/miniconda/bin/activate workshop_ngs && your_command_here"` block. This ensures the environment is properly sourced before the command runs. Let's try listing the environment contents and checking `CONDA_PREFIX` within that activated session:

In [7]:
!bash -c "source /usr/local/miniconda/etc/profile.d/conda.sh && conda activate workshop_ngs && \
          echo '--- Environment activated ---' && \
          echo 'CONDA_PREFIX in activated env: '$CONDA_PREFIX && \
          echo '--- Listing contents of activated env ---' && \
          conda list"

--- Environment activated ---
CONDA_PREFIX in activated env: 
--- Listing contents of activated env ---
# packages in environment at /usr/local/miniconda/envs/workshop_ngs:
#
# Name                       Version          Build                 Channel
_libgcc_mutex                0.1              main
_openmp_mutex                5.1              1_gnu
alsa-lib                     1.2.14           hb9d3cd8_0            conda-forge
bzip2                        1.0.8            hda65f42_8            conda-forge
c-ares                       1.34.5           hb9d3cd8_0            conda-forge
ca-certificates              2025.11.12       hbd8a1cb_0            conda-forge
cairo                        1.18.4           h3394656_0            conda-forge
certifi                      2024.8.30        pyhd8ed1ab_0          conda-forge
cutadapt                     4.4              py37h8902056_0        bioconda
dnaio                        0.10.0           py37h8902056_1        bioconda
fastqc      

We’ll be using real WGS resequencing data for this workshop. The dataset is a small subset for demonstration purposes. We will download the data from (https://zenodo.org/records/14258052) . Step 1 would be to make a parent directory to sort all the data in one place.


In [8]:
!mkdir -p fastq_files
!ls -F

fastq_files/  miniconda.sh  sample_data/


Now, let's navigate into the `fastq_files` directory.

In [9]:
!cd fastq_files/ && pwd

/content/fastq_files


Let's organize the download process by first ensuring the `fastq_files` directory exists, then navigating into it to download all the FASTQ files directly to their intended location.

In [14]:
# Define the list of FASTQ file links
fastq_links = [
    "https://zenodo.org/records/14258052/files/BEN_CI16_sub_1.fq.gz",
    "https://zenodo.org/records/14258052/files/BEN_CI16_sub_2.fq.gz",
    "https://zenodo.org/records/14258052/files/BEN_CI18_sub_1.fq.gz",
    "https://zenodo.org/records/14258052/files/BEN_CI18_sub_2.fq.gz",
    "https://zenodo.org/records/14258052/files/BEN_NW10_sub_1.fq.gz",
    "https://zenodo.org/records/14258052/files/BEN_NW10_sub_2.fq.gz",
    "https://zenodo.org/records/14258052/files/BEN_NW12_sub_1.fq.gz",
    "https://zenodo.org/records/14258052/files/BEN_NW12_sub_2.fq.gz",
    "https://zenodo.org/records/14258052/files/BEN_NW13_sub_1.fq.gz",
    "https://zenodo.org/records/14258052/files/BEN_NW13_sub_2.fq.gz",
    "https://zenodo.org/records/14258052/files/BEN_SI18_sub_1.fq.gz",
    "https://zenodo.org/records/14258052/files/BEN_SI18_sub_2.fq.gz",
    "https://zenodo.org/records/14258052/files/BEN_SI19_sub_1.fq.gz",
    "https://zenodo.org/records/14258052/files/BEN_SI19_sub_2.fq.gz",
    "https://zenodo.org/records/14258052/files/BEN_SI9_sub_1.fq.gz",
    "https://zenodo.org/records/14258052/files/BEN_SI9_sub_2.fq.gz",
    "https://zenodo.org/records/14258052/files/LGS1_sub_1.fq.gz",
    "https://zenodo.org/records/14258052/files/LGS1_sub_2.fq.gz"
]

# Create the directory if it doesn't exist, and then download files into it
!mkdir -p fastq_files

# Change to the directory and download files
# Using a subshell (bash -c) ensures the 'cd' command applies to the subsequent 'wget' commands
for link in fastq_links:
    !bash -c "cd fastq_files && wget -P . -q {link}"

# List the contents of the fastq_files directory to confirm all downloads
!ls -F fastq_files/

BEN_CI16_sub_1.fq.gz	BEN_NW12_sub_1.fq.gz	BEN_SI19_sub_1.fq.gz
BEN_CI16_sub_1.fq.gz.1	BEN_NW12_sub_1.fq.gz.1	BEN_SI19_sub_1.fq.gz.1
BEN_CI16_sub_2.fq.gz	BEN_NW12_sub_2.fq.gz	BEN_SI19_sub_2.fq.gz
BEN_CI16_sub_2.fq.gz.1	BEN_NW12_sub_2.fq.gz.1	BEN_SI19_sub_2.fq.gz.1
BEN_CI18_sub_1.fq.gz	BEN_NW13_sub_1.fq.gz	BEN_SI9_sub_1.fq.gz
BEN_CI18_sub_1.fq.gz.1	BEN_NW13_sub_1.fq.gz.1	BEN_SI9_sub_1.fq.gz.1
BEN_CI18_sub_2.fq.gz	BEN_NW13_sub_2.fq.gz	BEN_SI9_sub_2.fq.gz
BEN_CI18_sub_2.fq.gz.1	BEN_NW13_sub_2.fq.gz.1	BEN_SI9_sub_2.fq.gz.1
BEN_NW10_sub_1.fq.gz	BEN_SI18_sub_1.fq.gz	LGS1_sub_1.fq.gz
BEN_NW10_sub_1.fq.gz.1	BEN_SI18_sub_1.fq.gz.1	LGS1_sub_1.fq.gz.1
BEN_NW10_sub_2.fq.gz	BEN_SI18_sub_2.fq.gz	LGS1_sub_2.fq.gz
BEN_NW10_sub_2.fq.gz.1	BEN_SI18_sub_2.fq.gz.1	LGS1_sub_2.fq.gz.1


To visualise the content of the directory 'fastq_files' , to confirm that all the files have been successfully downloaded.

Once all the fastq files are downloaded, the next would be to run "FastQC" tool on the raw reads for the quality assessment step before the downstream analysis part.

***Why it's important:*** Poor-quality reads can lead to false variant calls or poor mapping, so quality control is crucial.

To do so, first, let's create a directory to store the FastQC reports.

In [17]:
!mkdir -p fastqc_results
!ls -F

fastqc_results/  fastq_files/  miniconda.sh  sample_data/


Now, we will run FastQC on all the FASTQ files within the `workshop_ngs` environment and save the reports to the `fastqc_results` directory.

In [18]:
%%time
!bash -c "source /usr/local/miniconda/etc/profile.d/conda.sh && conda activate workshop_ngs && \
          fastqc fastq_files/*.fq.gz -o fastqc_results"

# List the generated FastQC reports
!ls -F fastqc_results/

application/gzip
application/gzip
application/gzip
Started analysis of BEN_CI16_sub_1.fq.gz
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
Approx 5% complete for BEN_CI16_sub_1.fq.gz
Approx 10% complete for BEN_CI16_sub_1.fq.gz
Approx 15% complete for BEN_CI16_sub_1.fq.gz
Approx 20% complete for BEN_CI16_sub_1.fq.gz
Approx 25% complete for BEN_CI16_sub_1.fq.gz
Approx 30% complete for BEN_CI16_sub_1.fq.gz
Approx 35% complete for BEN_CI16_sub_1.fq.gz
Approx 40% complete for BEN_CI16_sub_1.fq.gz
Approx 45% complete for BEN_CI16_sub_1.fq.gz
Approx 50% complete for BEN_CI16_sub_1.fq.gz
Approx 55% complete for BEN_CI16_sub_1.fq.gz
Approx 60% complete for BEN_CI16_sub_1.fq.gz
Approx 65% complete for BEN_CI16_sub_1.fq.gz
Approx 70% complete for BEN_CI16_sub_1.fq.gz
Approx 75% complete for 

The .html files generated by FastQC are standard web pages, which means you can visualize them by opening them in any web browser!

Here's how you can access them from Colab:
 On the left-hand side of your Colab interface, there's usually a file explorer icon (a folder). Click on it to open the file browser. Navigate to the fastqc_results directory. You'll see all the .html files listed there. You can right-click on any .html file and select 'Download' to save it to your local machine, then open it with your preferred web browser.




Post qualitty assessment, the next step will be the trimming of the adapters or low-quality bases using tools such as Trim-galore. This step will ensure producing clean FASTQ files, ready for mapping.

Step 1: We'll create a directory to store the trimmed FASTQ files.

In [19]:
!mkdir -p trimmed_fastq_files
!ls -F

fastqc_results/  fastq_files/  miniconda.sh  sample_data/  trimmed_fastq_files/


Now, we will activate the `workshop_ngs` conda environment and run `Trim Galore!` on all the FASTQ files. We'll specify that they are paired-end reads and direct the output to the `trimmed_fastq_files` directory.

Trim Galore automatically detects and removes common adapter sequences and performs quality trimming. The `--paired` option is essential for correctly processing paired-end reads.

In [22]:
%%time
# Run Trim Galore! within the activated conda environment
# We iterate through the samples to process paired-end reads correctly

# Get a list of fastq files
fastq_files_list = !ls fastq_files/*.fq.gz

# Extract unique sample prefixes (e.g., BEN_CI16, BEN_CI18) from the filenames
# We split by '/' to get just the filename, then split by '_sub_' to get the sample prefix
sample_prefixes = sorted(list(set([f.split('/')[-1].split('_sub_')[0] for f in fastq_files_list])))

for prefix in sample_prefixes:
    read1 = f"fastq_files/{prefix}_sub_1.fq.gz"
    read2 = f"fastq_files/{prefix}_sub_2.fq.gz"
    output_dir = "trimmed_fastq_files"

    # Use bash -c to activate conda env and run trim_galore
    # --paired: for paired-end reads
    # -o: specify output directory
    # --fastqc: run FastQC on the trimmed output (optional but good for QC after trimming)
    !bash -c "source /usr/local/miniconda/etc/profile.d/conda.sh && conda activate workshop_ngs && \
              trim_galore --paired {read1} {read2} -o {output_dir} --fastqc"

# List the contents of the trimmed_fastq_files directory to confirm
!ls -F trimmed_fastq_files/

Multicore support not enabled. Proceeding with single-core trimming.
Path to Cutadapt set as: 'cutadapt' (default)
Cutadapt seems to be working fine (tested command 'cutadapt --version')
Cutadapt version: 4.4
single-core operation.
igzip command line interface 2.31.1
igzip detected. Using igzip for decompressing

No quality encoding type selected. Assuming that the data provided uses Sanger encoded Phred scores (default)

Output will be written into the directory: /content/trimmed_fastq_files/


AUTO-DETECTING ADAPTER TYPE
Attempting to auto-detect adapter type from the first 1 million sequences of the first file (>> fastq_files/BEN_NW13_sub_1.fq.gz <<)

Found perfect matches for the following adapter sequences:
Adapter type	Count	Sequence	Sequences analysed	Percentage
Illumina	43421	AGATCGGAAGAGC	1000000	4.34
Nextera	14	CTGTCTCTTATA	1000000	0.00
smallRNA	6	TGGAATTCTCGG	1000000	0.00
Using Illumina adapter for trimming (count: 43421). Second best hit was Nextera (count: 14)

Writing rep